# Pandas for Data Science
This notebook demonstrates powerful capabilities of the Pandas library using a complex dataset.


In [ ]:
import pandas as pd
import numpy as np

# load employee data
employee_data = pd.read_csv("employee_data.csv")
# Display the first few rows of the dataset
employee_data.head()

### GroupBy and Multi-Level Aggregation

In [ ]:
# Average salary and performance by Department and Team
agg_employee_data = employee_data.groupby(["Department", "Team"]).agg(
    {
        "Salary": ["mean", "max", "min"],
        "PerformanceScore": ["mean", "std"],
        "Age": "median",
    }
)
agg_employee_data

### Filtering with Complex Conditions

In [ ]:
# Employees in Engineering, older than 40, with high performance
filtered = employee_data[
    (employee_data["Department"] == "Engineering")
    & (employee_data["Age"] > 40)
    & (employee_data["PerformanceScore"] > 3.5)
]
filtered.sort_values(by="PerformanceScore", ascending=False)

### Using `apply()` with Custom Logic

In [ ]:
def experience_level(row):
    years = 2025 - pd.to_datetime(row["JoinDate"]).year
    if years < 2:
        return "Junior"
    elif years < 5:
        return "Mid"
    else:
        return "Senior"


employee_data["ExperienceLevel"] = employee_data.apply(experience_level, axis=1)
employee_data[["Name", "JoinDate", "ExperienceLevel"]].head()

### Creating Pivot Tables

In [ ]:
pivot = pd.pivot_table(
    employee_data,
    index="Department",
    columns="ExperienceLevel",
    values="Salary",
    aggfunc="mean",
)
pivot

### Merging DataFrames

In [ ]:
# Simulate a new DataFrame with bonus info
bonus_employee_data = pd.DataFrame(
    {
        "EmployeeID": np.random.choice(
            employee_data["EmployeeID"], size=50, replace=False
        ),
        "Bonus": np.random.randint(1000, 10000, size=50),
    }
)

# Merge with main data
employee_data_merged = employee_data.merge(
    bonus_employee_data, on="EmployeeID", how="left"
)
employee_data_merged[["Name", "Department", "Bonus"]].head()

###  Time Series: Resampling and Rolling Averages

In [ ]:
# Convert to datetime and set index in one step
monthly_joins = (
    employee_data.set_index(pd.to_datetime(employee_data["JoinDate"]))
    .resample("M")["EmployeeID"]
    .count()
)
monthly_joins.plot(title="Monthly New Joiners", figsize=(10, 4));

## Exercice: Sales Data Analysis

You are provided with a dataset `sales_data.csv` containing retail transactions with the following columns:

- `OrderID`: Order identifier  
- `CustomerID`: Unique customer identifier  
- `Product`: Product name  
- `Category`: Product category  
- `Quantity`: Number of units sold  
- `Price`: Unit price  
- `OrderDate`: Date of the order  
- `Country`: Customer's country  

### Tasks:

1. **Basic Exploration**:
   - Display the first 5 rows.
   - Count missing values in each column.
   - Print summary statistics for `Quantity` and `Price`.

2. **Data Cleaning**:
   - Remove rows where `Quantity` or `Price` is less than or equal to 0.
   - Convert `OrderDate` to datetime format.

3. **Feature Engineering**:
   - Create a new column `TotalAmount` = `Quantity` × `Price`.
   - Extract the `Month` and `DayOfWeek` from `OrderDate`.

4. **Analysis**:
   - What is the total revenue per `Country`? (Sort descending)
   - What are the top 5 best-selling products by `Quantity`?
   - How many unique customers are there in each `Country`?

5. **Time Series Aggregation**:
   - Plot total daily revenue (`OrderDate` vs. `TotalAmount`) using a line plot.
   - Compute the monthly average quantity sold per category.

> 💡 Use `.groupby()`, `.agg()`, `.pivot_table()`, and visualization functions where appropriate.

In [ ]:
# 1. Basic Exploration
sales_df = pd.read_csv("sales_data.csv")
print(sales_df.head())
print(sales_df[["Quantity", "Price"]].describe())

In [ ]:
# 2. Data Cleaning
sales_df = sales_df[(sales_df["Quantity"] > 0) & (sales_df["Price"] > 0)]
sales_df["OrderDate"] = pd.to_datetime(sales_df["OrderDate"])
sales_df

In [ ]:
# 3. Feature Engineering
sales_df["TotalAmount"] = sales_df["Quantity"] * sales_df["Price"]
sales_df["Month"] = pd.to_datetime(sales_df["OrderDate"]).dt.month
sales_df["DayOfWeek"] = pd.to_datetime(sales_df["OrderDate"]).dt.dayofweek

In [ ]:
# 4. Analysis
total_revenue_per_country = (
    sales_df.groupby("Country")
    .agg({"TotalAmount": "sum"})
    .sort_values(by="TotalAmount", ascending=False)
)
print(total_revenue_per_country)

top_5_products = (
    sales_df.groupby("Product").agg({"Quantity": "sum"}).nlargest(5, "Quantity")
)
print(top_5_products)

unique_customer_per_country = (
    sales_df.groupby("Country")
    .agg({"CustomerID": "nunique"})
    .sort_values(by="CustomerID", ascending=False)
)
print(unique_customer_per_country)

In [ ]:
# 5. Time Series Aggregation
import matplotlib.pyplot as plt

daily_revenue = sales_df.groupby("OrderDate")["TotalAmount"].sum()
daily_revenue.plot(
    kind="line",
    title="Total Daily Revenue",
    figsize=(12, 6),
    xlabel="Order Date",
    ylabel="Total Revenue",
)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Compute monthly average quantity sold per category
sales_df["YearMonth"] = sales_df["OrderDate"].dt.to_period("M")
monthly_avg_quantity = (
    sales_df.groupby(["YearMonth", "Category"])["Quantity"].mean().unstack(fill_value=0)
)

print("Monthly Average Quantity Sold per Category:")
print(monthly_avg_quantity)